# Data Project 2021: School performance across Danish municipalities 

The aim of the data project is to investigate primary school performance across Danish municipalities based on the grade point average obtained by the students at the final exam. We fetch data for performance of primary schools from the Danish Ministry of Children and Education. Futhermore, we fetch data from Statistics Denmark covering average income levels and shares of hihgly educated inhabitants in the Danish muncipalities. First, we read and clean the data sets in order to merge them into a single DataFrame. Then, we analyze the data and investigate whether there is positive correlation between income and education levels and the performance of primary schools across municipalities. 

**The project is organized as follows:**
1. Data project description
2. Reading and cleaning data 
3. Merging data 
4. Data analysis
6. Conclusion

**Group:** SSTL

**Members:** Louise Otte Arildsen (srb330), Signe Kolind (hmk792), Stine Fürst (qvj635) & Thomas Nielsen (qzw392)

**Imports and set magics:**

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib_venn import venn2 # install with pip install matplotlib-venn

import pandas_datareader #!pip install pandas-datareader
import pydst #!pip install git+https://github.com/elben10/pydst

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

## School performance data

**Reading and cleaning the data**<br>
We read the school data in ``Skoletal.xlsx`` obtained from [education statistics](https://uddannelsesstatistik.dk/Pages/Reports/1834.aspx) conducted by the Danish Ministry of Children and Education. The data set contains key indicators of performance for the primary schools in Denmark at municpality level. 
We clean the data by removing redundant columns and renaming the columns:

In [3]:
# a. load
school = pd.read_excel('Skoletal.xlsx', skiprows=2)

# b. drop columns
drop_these = ['Rækkenavne', 'Overgang 15 mdr ungdomsuddannelse fra 9 og 10 kl', 'Signifikant forskel', 'Forskel', 'Kompetencedækning']
school.drop(drop_these, axis=1, inplace=True)

# c. rename columns
school.rename(columns = {'Institution':'school', 'Skoleår':'year', 'Karaktergennemsnit_skoletal':'gpa', 'Socioøkonomisk reference':'socioeconomic', 'Andel med højest trivsel':'well-being', 'Samlet elevfravær':'absence', 'Elevtal':'students', 'Klassekvotient':'class'}, inplace=True)

The dataset now looks like this:

In [4]:
school.head()
#print(school.tail())

,school,year,gpa,socioeconomic,well-being,absence,students,class
0,Den Classenske Legatskole,2013/2014,8.2,7.8,NaN,NaN,500.0,24.95
1,Den Classenske Legatskole,København,8.2,7.8,NaN,NaN,500.0,24.95
2,Den Classenske Legatskole,2014/2015,7.6,7.5,0.906615,0.089829,521.0,24.90
3,Den Classenske Legatskole,København,7.6,7.5,0.906615,0.089829,521.0,24.90
4,Den Classenske Legatskole,2015/2016,8.2,8.0,0.956044,0.079799,515.0,24.90


**Seperate municipality and school year in column 'Year'**:<br>
In the data set every row is duplicated such that the data is the same in every column except from in column 'Year'. The column 'Year' alternately displays school year and municipality and we want to split this into two seperate columns. 

In [5]:
#First we check the numbers of rows in the data set school:
print(f'The number of rows in data set school is = {len(school)}')

#Then we create a new variable 'municipality', which extract allmuncipality names from 'year'
municipality=pd.DataFrame(school['year'].iloc[1::2].values)

#We check that the number of municipality names are equal to half of the row lines. 
print(f'31328 divided by 2 = {len(school)/2}')
print(f'The number of rows in data set municipality is = {len(municipality)}')

#Then we delete every other ro (then ones containing municipalities in the column 'year'
res = [True, False] * 15664
school_new=school[res]
school_new.reset_index(drop=True, inplace=True)
print(f'The number of rows in data set school_new is = {len(school_new)}')

#We then merge school_new with municipalities and arrive at the solution
school_final = pd.concat([school_new, municipality], axis=1)
school_final.tail()
school_final.rename(columns = {school_final.columns[8]:'municipality'}, inplace=True)
school_final.head()

The number of rows in data set school is = 31328
31328 divided by 2 = 15664.0
The number of rows in data set municipality is = 15664
The number of rows in data set school_new is = 15664


,school,year,gpa,socioeconomic,well-being,absence,students,class,municipality
0,Den Classenske Legatskole,2013/2014,8.2,7.8,NaN,NaN,500.0,24.95,København
1,Den Classenske Legatskole,2014/2015,7.6,7.5,0.906615,0.089829,521.0,24.90,København
2,Den Classenske Legatskole,2015/2016,8.2,8.0,0.956044,0.079799,515.0,24.90,København
3,Den Classenske Legatskole,2016/2017,7.8,7.8,0.913669,0.072769,509.0,25.60,København
4,Den Classenske Legatskole,2017/2018,8.8,8.1,0.919540,0.081672,498.0,24.80,København


In [6]:
school_final.tail()

,school,year,gpa,socioeconomic,well-being,absence,students,class,municipality
15659,Vestermarkskolen Aars,2015/2016,NaN,NaN,1.0,0.094875,49.0,NaN,Vesthimmerlands
15660,Vestermarkskolen Aars,2016/2017,NaN,NaN,1.0,0.089781,50.0,NaN,Vesthimmerlands
15661,Vestermarkskolen Aars,2017/2018,NaN,NaN,NaN,NaN,49.0,NaN,Vesthimmerlands
15662,Vestermarkskolen Aars,2018/2019,NaN,NaN,NaN,0.082224,46.0,NaN,Vesthimmerlands
15663,Vestermarkskolen Aars,2019/2020,NaN,NaN,NaN,0.085611,47.0,NaN,Vesthimmerlands


## Income level data

**Reading and cleaning the data**<br>
* Explain where the data is fetched from and how. 
* Briefly explain what the data contains
* Explain how you clean 

In [7]:
Dst = pydst.Dst(lang='en') #

In [8]:
Dst.get_subjects()

,id,desc,active,hasSubjects
0,02,Population and elections,True,True
1,04,"Labour, income and wealth",True,True
2,06,Prices and consumption,True,True
3,05,Living conditions,True,True
4,14,National accounts and government finances,True,True
5,03,Education and knowledge,True,True
6,11,Business sectors,True,True
7,07,Business sector in general,True,True
8,16,Money and credit market,True,True
9,13,External economy,True,True


In [9]:
tables = Dst.get_tables(subjects=['04'])
tables
tables[tables.id == 'INDKP101']

,id,text,unit,updated,firstPeriod,latestPeriod,active,variables
194,INDKP101,People,-,2020-11-27 08:00:00,1987,2019,True,"[region, unit, sex, type of income, time]"


In [10]:
indk_var = Dst.get_variables(table_id='INDKP101')
indk_var

indk_var = Dst.get_variables(table_id='INDKP101')
for id in ['OMRÅDE','ENHED','KOEN','INDKOMSTTYPE', 'Tid']:
    print(id)
    values = indk_var.loc[indk_var.id == id,['values']].values[0,0]
    for value in values:      
        print(f' id = {value["id"]}, text = {value["text"]}')

OMRÅDE
 id = 000, text = All Denmark
 id = 01, text = Province Byen København
 id = 101, text = Copenhagen
 id = 147, text = Frederiksberg
 id = 155, text = Dragør
 id = 185, text = Tårnby
 id = 02, text = Province Københavns omegn
 id = 165, text = Albertslund
 id = 151, text = Ballerup
 id = 153, text = Brøndby
 id = 157, text = Gentofte
 id = 159, text = Gladsaxe
 id = 161, text = Glostrup
 id = 163, text = Herlev
 id = 167, text = Hvidovre
 id = 169, text = Høje-Taastrup
 id = 183, text = Ishøj
 id = 173, text = Lyngby-Taarbæk
 id = 175, text = Rødovre
 id = 187, text = Vallensbæk
 id = 03, text = Province Nordsjælland
 id = 201, text = Allerød
 id = 240, text = Egedal
 id = 210, text = Fredensborg
 id = 250, text = Frederikssund
 id = 190, text = Furesø
 id = 270, text = Gribskov
 id = 260, text = Halsnæs
 id = 217, text = Helsingør
 id = 219, text = Hillerød
 id = 223, text = Hørsholm
 id = 230, text = Rudersdal
 id = 04, text = Province Bornholm
 id = 400, text = Bornholm
 id = 

In [11]:
variables = {'OMRÅDE':['*'],'ENHED':['116'],'KOEN':['MOK'],'INDKOMSTTYPE':['105'], 'Tid':['2013','2014','2015','2016','2017','2018','2019']}
inc_api = Dst.get_data(table_id = 'INDKP101', variables=variables)
display(inc_api)

,OMRÅDE,ENHED,KOEN,INDKOMSTTYPE,TID,INDHOLD
0,Assens,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2013,270534
1,Faaborg-Midtfyn,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2013,267375
2,Kerteminde,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2013,276207
3,Nyborg,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2013,270759
4,Odense,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2013,266297
...,...,...,...,...,...,...
765,Norddjurs,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2019,294950
766,Thisted,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2019,306245
767,Kerteminde,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2019,319218
768,Varde,Average income for all people (DKK),"Men and women, total","2 Pre-tax Income, total (3+7+22+26+29)",2019,314202


In [12]:
inc_1=inc_api.drop(['KOEN','INDKOMSTTYPE','ENHED'], axis=1)

inc_1.rename(columns = {'INDHOLD':'avg_income','OMRÅDE':'municipality','TID':'year'}, inplace=True)
display(inc_1)

print(f'Years in dat: {inc_1.year.unique()}')
print(f'Municipalities in dat = {len(inc_1.municipality.unique())}')

,OMRÅDE,TID,INDHOLD
0,Assens,2013,270534
1,Faaborg-Midtfyn,2013,267375
2,Kerteminde,2013,276207
3,Nyborg,2013,270759
4,Odense,2013,266297
...,...,...,...
765,Norddjurs,2019,294950
766,Thisted,2019,306245
767,Kerteminde,2019,319218
768,Varde,2019,314202


,municipality,year,avg_income
0,Assens,2013,270534
1,Faaborg-Midtfyn,2013,267375
2,Kerteminde,2013,276207
3,Nyborg,2013,270759
4,Odense,2013,266297
...,...,...,...
765,Norddjurs,2019,294950
766,Thisted,2019,306245
767,Kerteminde,2019,319218
768,Varde,2019,314202


Years in dat: [2013 2014 2015 2016 2017 2018 2019]
Municipalities in dat = 110


## Education level data

**Reading and cleaning the data**<br>
* Explain where the data is fetched from and how. 
* Briefly explain what the data contains
* Explain how you clean 


In [13]:
Louises data

SyntaxError: invalid syntax (<ipython-input-13-d320e25a9b3a>, line 1)

## Explore data set

In order to be able to **explore the raw data**, we here provide an **interactive plot** to show, respectively, the employment and income level in each municipality

The **static plot** is:

In [ ]:
def plot_empl_inc(school,dataset,municipality): 
    
    dataset == 'School'
    df = school_final
    y = 'employment'

    
    I = df['municipality'] == municipality
    ax = df.loc[I,:].plot(x='year', y=y, style='-o')

The **interactive plot** is:

ADD SOMETHING HERE IF THE READER SHOULD KNOW THAT E.G. SOME MUNICIPALITY IS SPECIAL.

# Merge data sets

We now create a data set with **municpalities which are in both of our data sets**. We can illustrate this **merge** as:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('inc', 'empl'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

In [ ]:
merged = pd.merge(empl_long, inc_long, how='inner',on=['municipality','year'])

print(f'Number of municipalities = {len(merged.municipality.unique())}')
print(f'Number of years          = {len(merged.year.unique())}')

# Analysis

To get a quick overview of the data, we show some **summary statistics by year**:

In [ ]:
.groupby('year').agg(['mean','std']).round(2)
titanic.groupby(["Sex", "Pclass"])["Fare"].mean()

ADD FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.